# Showcase of Bybit Future - Future calc
The BybitClient has custom methods that I coded, but can also make normal request to the Bybit API with the session attribute.

In [3]:
from Bybit.bybit import BybitClient
from Bybit.utils import plot_candles

# Create a Bybit client
client = BybitClient()

# Get all futures BTC gaps
client.all_gaps()

,Buy,Sell,Gap,Coeff,APR,DaysLeft,CumVolume
0,BTC-18OCT24,BTC-25OCT24,$ 96.50,0.15 %,5.28 %,5,363.99K
1,BTC-18OCT24,BTC-01NOV24,$ 178.50,0.28 %,9.74 %,5,242.02K
2,BTC-18OCT24,BTC-29NOV24,$ 529.00,0.84 %,28.90 %,5,235.97K
3,BTC-18OCT24,BTC-27DEC24,$ 928.50,1.47 %,50.72 %,5,490.18K
4,BTC-18OCT24,BTC-28MAR25,$ 2219.00,3.51 %,121.25 %,5,820.77K
5,BTC-18OCT24,BTC-27JUN25,$ 3649.00,5.78 %,199.37 %,5,700.24K
6,BTC-18OCT24,BTC-26SEP25,$ 5323.00,8.42 %,290.83 %,5,398.58K
7,BTC-25OCT24,BTC-01NOV24,$ 82.00,0.13 %,1.93 %,12,227.83K
8,BTC-25OCT24,BTC-29NOV24,$ 432.50,0.68 %,10.15 %,12,221.78K
9,BTC-25OCT24,BTC-27DEC24,$ 832.00,1.31 %,19.53 %,12,475.98K


In [4]:
plot_candles("BTC-26SEP25_3.json").show()